In [1]:
# jap2010を読み込んでparquetにするcode｡実行不要

"""
テキストアーカイブ - 日本語ウェブコーパス 2010
http://www.s-yata.jp/corpus/nwc2010/texts/
を処理するcode｡

#URLは常時は空いていないので注意｡(畠山が持ってます)
wget http://www.s-yata.jp/corpus/nwc2010/texts/filelist
wget -i filelist
"""

'\nテキストアーカイブ - 日本語ウェブコーパス 2010\nhttp://www.s-yata.jp/corpus/nwc2010/texts/\nを処理するcode｡\n\n#URLは常時は空いていないので注意｡(畠山が持ってます)\nwget http://www.s-yata.jp/corpus/nwc2010/texts/filelist\nwget -i filelist\n'

In [2]:
import lzma
import glob
import json
from tqdm import tqdm
import pandas as pd
task_name="00000001"
path=f"data/jap2010/{task_name}.xz"
# .xz ファイルを開いて内容をチャンクで読み込む
with lzma.open(path, 'rt') as file:
    lines=file.readlines()

In [4]:

out_dir="data/jap2010_parquet"
new_line_count=0
count_threshold=3
current_doc=[]
cnt=0
text_len=0
text_list=[]
for line in (lines):
    if line=="\n":
        new_line_count+=1
    else:
        new_line_count=0
    current_doc.append(line)
    if new_line_count>=count_threshold:
        text="\n".join(current_doc)
        text=text.strip()
        text=text.replace("\n\n","\n")
        current_doc=[]
        new_line_count=0

        """
        if text!="":
            d=json.dumps({"text":text},ensure_ascii=False)
            with open(f"{out_dir}/{task_name}.jsonl","a") as f:
                f.write(f"{d}\n")
        """
        if text!="":
            text_list.append(text)
            text_len+=len(text)

    if text_len>10**8:
        df=pd.DataFrame(text_list,columns=["text"])
        df.to_parquet(f"{out_dir}/{task_name}_{cnt}.parquet")
        cnt+=1
        text_list=[]
        text_len=0
        #if cnt>100:
        #    break
df=pd.DataFrame(text_list,columns=["text"])
df.to_parquet(f"{out_dir}/{task_name}_{cnt}.parquet")

In [42]:
#huggingface-cli upload --repo-type=dataset hatakeyama-llm-team/japanese2010 .